<a href="https://colab.research.google.com/github/rbawden/Tutoriel-Normalisation/blob/main/Tutoriel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installer les paquets nécessaires

In [19]:
!pip install gdown==4.2.0 
!pip install fairseq@git+git://github.com/pytorch/fairseq.git@5a75b079bf8911a327940c28794608e003a9fa52 
!pip install sentencepiece sacrebleu hydra-core
!pip install omegaconf==2.0.5

  Using cached fairseq-1.0.0a0+5a75b07-cp37-cp37m-macosx_10_9_x86_64.whl
  Using cached omegaconf-2.0.6-py3-none-any.whl (36 kB)
  Attempting uninstall: omegaconf
    Found existing installation: omegaconf 1.4.1
    Uninstalling omegaconf-1.4.1:
      Successfully uninstalled omegaconf-1.4.1


  Attempting uninstall: omegaconf
    Found existing installation: omegaconf 2.0.6
    Uninstalling omegaconf-2.0.6:
      Successfully uninstalled omegaconf-2.0.6


In [18]:
!python --version
import fairseq
fairseq.__version__

Python 3.7.11


ImportError: cannot import name 'II' from 'omegaconf' (/Users/rbawden/Library/Python/3.8/lib/python/site-packages/omegaconf/__init__.py)

Télécharger les données et les modèles depuis Google Drive

In [3]:
![ -d data-models ] || gdown https://drive.google.com/drive/folders/1h-qSnPBPZFZQ_kqWIBMhkkFS-6C2b10H?usp=sharing -O data-models --folder 

Créer des liens symboliques pour rendre les fichiers/dossiers téléchargés plus accessible

In [4]:
![ -d data ] || mkdir data; [ -d models ] || mkdir models
!for file in data-models/*{src,trg,model,vocab}; do ln -sf  $PWD/$file $PWD/data/; done
!for file in data-models/*{txt,npz}; do ln -sf $PWD/$file $PWD/models/; done

In [5]:
import os, sys
for folder in 'models', 'data':
  path = os.path.abspath(os.path.join('./', folder))
  if path not in sys.path:
    sys.path.append(path)

Fonction pour lire le contenu d'un fichier ligne par ligne

In [6]:
def lire_fichier(nom_fichier):
  contenu = []
  with open(nom_fichier) as fp:
    for line in fp:
      contenu.append(line.strip())
  return contenu

def ecrire_fichier(liste_phrases, nom_fichier):
    with open(nom_fichier, 'w') as fp:
        for phrase in liste_phrases:
            fp.write(phrase + '\n')

Lire le contenu des données parallèles

In [7]:
data_src = lire_fichier(os.getcwd() + '/data/dev.src')
data_trg = lire_fichier('data/dev.trg')

Visualiser les fichiers sources et cibles

In [8]:
for i in range(4):
    print('src = ', data_src[i])
    print('trg = ', data_trg[i])
    print('--')

src =  1.
trg =  1.
--
src =  1. QVe cette propoſtion, qu'vn eſpace eſt vuidé, repugne au ſens commun.
trg =  1. QUe cette proposition, qu'un espace est vidé, répugne au sens commun.
--
src =  1. QVe tous les corps ont repugnance à ſe ſeparer l'vn de l'autre, & admettre du vuide dans leur interualle;
trg =  1. QUe tous les corps ont répugnance à se séparer l'un de l'autre, et admettre du vide dans leur intervalle;
--
src =  1. QVe tous les corps ont repugnance à ſe ſeparer l'vn de l'autre, & admettre ce vuide apparent dans leur interualle:
trg =  1. QUe tous les corps ont répugnance à se séparer l'un de l'autre, et admettre ce vide apparent dans leur intervalle:
--


Charger le modèle de segmentation en sous-mots

In [9]:
import sentencepiece
spm = sentencepiece.SentencePieceProcessor(model_file='data/bpe_joint_1000.model')

Appliquer le modèle sur les données

In [10]:
data_src_sp = spm.encode(data_src, out_type=str)
data_trg_sp = spm.encode(data_trg, out_type=str)

Ecrire les données pre-traités dans des fichiers

Visualiser les fichiers

In [11]:
ecrire_fichier([' '.join(phrase) for phrase in data_src_sp], 'data/dev.sp.src')
ecrire_fichier([' '.join(phrase) for phrase in data_trg_sp], 'data/dev.sp.trg')

In [12]:
for i in range(4):
    print('src = ', data_src_sp[i])
    print('trg = ', data_trg_sp[i])
    print('--')

src =  ['▁1', '.']
trg =  ['▁1', '.']
--
src =  ['▁1', '.', '▁Q', 'V', 'e', '▁cette', '▁prop', 'ost', 'ion', ',', '▁qu', "'", 'vn', '▁esp', 'ace', '▁est', '▁v', 'ui', 'd', 'é', ',', '▁re', 'p', 'u', 'gne', '▁au', '▁sens', '▁comm', 'un', '.']
trg =  ['▁1', '.', '▁Q', 'U', 'e', '▁cette', '▁prop', 'os', 'it', 'ion', ',', '▁qu', "'", 'un', '▁esp', 'ace', '▁est', '▁v', 'id', 'é', ',', '▁rép', 'u', 'gne', '▁au', '▁sens', '▁comm', 'un', '.']
--
src =  ['▁1', '.', '▁Q', 'V', 'e', '▁tous', '▁les', '▁cor', 'p', 's', '▁ont', '▁re', 'p', 'u', 'gn', 'ance', '▁à', '▁se', '▁se', 'p', 'are', 'r', '▁l', "'", 'vn', '▁de', '▁l', "'", 'autre', ',', '▁&', '▁ad', 'm', 'ettre', '▁du', '▁v', 'ui', 'de', '▁dans', '▁leur', '▁in', 'ter', 'u', 'al', 'le', ';']
trg =  ['▁1', '.', '▁Q', 'U', 'e', '▁tous', '▁les', '▁cor', 'p', 's', '▁ont', '▁rép', 'u', 'gn', 'ance', '▁à', '▁se', '▁s', 'ép', 'are', 'r', '▁l', "'", 'un', '▁de', '▁l', "'", 'autre', ',', '▁et', '▁ad', 'm', 'ettre', '▁du', '▁vi', 'de', '▁dans', '▁leur', 

Définir une fonction pour détokeniser (pour plus tard)

In [14]:
def decode_sp(list_text):
    return [''.join(sent).replace('▁', ' ') for sent in list_text]

In [15]:
decode_sp(data_src_sp[:5])

[' 1.',
 " 1. QVe cette propostion, qu'vn espace est vuidé, repugne au sens commun.",
 " 1. QVe tous les corps ont repugnance à se separer l'vn de l'autre, & admettre du vuide dans leur interualle;",
 " 1. QVe tous les corps ont repugnance à se separer l'vn de l'autre, & admettre ce vuide apparent dans leur interualle:",
 ' 2.']

Charger le modèle de normalisation déjà entraîné

In [25]:
!cat data/dev.sp.src | fairseq-interactive models/ --source-lang src --target-lang trg --path models/model.pt > data/dev.sp.norm.trg.output

/Users/rbawden/miniconda3/envs/py37/lib/python3.7/site-packages/fairseq/search.py:140: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beams_buf = indices_buf // vocab_size
/Users/rbawden/miniconda3/envs/py37/lib/python3.7/site-packages/fairseq/sequence_generator.py:669: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  unfin_idx = idx // b

La sortie de fairseq-interactive donne quelque chose comme ceci:

```
S-0     ▁1 .
W-0     0.064   seconds
H-0     -0.00011481383990030736 ▁1 .
D-0     -0.00011481383990030736 ▁1 .
P-0     -0.0000 -0.0003 -0.0000
S-1     ▁1 . ▁Q V e ▁cette ▁prop ost ion , ▁qu ' vn ▁esp ace ▁est ▁v ui d é , ▁re p u gne ▁au ▁sens ▁comm un .
W-1     0.456   seconds
H-1     -0.039981111884117126   ▁1 . ▁Q U e ▁cette ▁prop ost ion , ▁qu ' un ▁esp ace ▁est ▁v ui d é , ▁rép u gne ▁au ▁sens ▁comm un .
D-1     -0.039981111884117126   ▁1 . ▁Q U e ▁cette ▁prop ost ion , ▁qu ' un ▁esp ace ▁est ▁v ui d é , ▁rép u gne ▁au ▁sens ▁comm un .
P-1     -0.0000 -0.0000 -0.0043 -0.0632 -0.0006 -0.0000 -0.0001 -0.9353 -0.0001 -0.0012 -0.0000 0.0000 -0.0001 -0.0078 -0.0070 -0.0000 -0
.0022 -0.1168 -0.0001 -0.0000 -0.0000 -0.0389 -0.0157 -0.0053 -0.0000 -0.0000 -0.0001 -0.0000 -0.0004 -0.0000
S-2     ▁1 . ▁Q V e ▁tous ▁les ▁cor p s ▁ont ▁re p u gn ance ▁à ▁se ▁se p are r ▁l ' vn ▁de ▁l ' autre , ▁& ▁ad m ettre ▁du ▁v ui de ▁dan
s ▁leur ▁in ter u al le ;
W-2     0.688   seconds
H-2     -0.019450930878520012   ▁1 . ▁Q U e ▁tous ▁les ▁cor p s ▁ont ▁rép u gn ance ▁à ▁se ▁s ép are r ▁l ' un ▁de ▁l ' autre , ▁et ▁ad m
 ettre ▁du ▁v ui de ▁dans ▁leur ▁in ter v és le ;
D-2     -0.019450930878520012   ▁1 . ▁Q U e ▁tous ▁les ▁cor p s ▁ont ▁rép u gn ance ▁à ▁se ▁s ép are r ▁l ' un ▁de ▁l ' autre , ▁et ▁ad m
 ettre ▁du ▁v ui de ▁dans ▁leur ▁in ter v és le ;
P-2     -0.0000 -0.0001 -0.0040 -0.1684 -0.0004 -0.0000 -0.0000 -0.0000 -0.0007 -0.0000 -0.0001 -0.1220 -0.0063 -0.0002 -0.0137 -0.0000 -
0.0000 -0.0002 -0.0001 -0.0248 -0.0022 -0.0003 -0.0000 -0.0000 -0.0000 -0.0000 -0.0000 -0.0000 -0.0002 -0.0001 -0.0000 -0.0000 -0.0000 -0
.0000 -0.0383 -0.0173 -0.0006 -0.0000 -0.0000 -0.0000 -0.0066 -0.0016 -0.4856 -0.0007 -0.0002 -0.0000
```

In [35]:
def get_outputs(filename):
    outputs = []
    with open(filename) as fp:
        for line in fp:
            if 'H-' in line:
                outputs.append(line.strip().split('\t')[2])
    return outputs

In [36]:
outputs = get_outputs('data/dev.sp.norm.trg')

In [37]:
outputs[:10]

['▁1 .',
 "▁1 . ▁Q U e ▁cette ▁prop ost ion , ▁qu ' un ▁esp ace ▁est ▁v ui d é , ▁rép u gne ▁au ▁sens ▁comm un .",
 "▁1 . ▁Q U e ▁tous ▁les ▁cor p s ▁ont ▁rép u gn ance ▁à ▁se ▁s ép are r ▁l ' un ▁de ▁l ' autre , ▁et ▁ad m ettre ▁du ▁v ui de ▁dans ▁leur ▁in ter v és le ;",
 "▁1 . ▁Q U e ▁tous ▁les ▁cor p s ▁ont ▁rép u gn ance ▁à ▁se ▁s ép are r ▁l ' un ▁de ▁l ' autre , ▁et ▁ad m ettre ▁ce ▁v ui de ▁app ar ent ▁dans ▁leur ▁in ter v és le :",
 '▁ 2 .',
 "▁ 2 . ▁Que ▁cette ▁h or re ur ▁ou ▁rép u gn ance ▁qu ' ont ▁tous ▁les ▁cor p s , ▁n ' est ▁pas ▁plus ▁grande ▁pour ▁ad m ettre ▁un ▁grand ▁v ui de , ▁qu ' un ▁petit :",
 "▁ 2 . ▁Que ▁cette ▁h or re ur ▁où ▁cette ▁rép u gn ance ▁qu ' ont ▁tous ▁les ▁cor p s ▁n ' est ▁pas ▁plus ▁grande ▁pour ▁ad m ettre ▁un ▁grand ▁v ui de ▁app ar ent ▁qu ' un ▁petit :",
 "▁ 2 . ▁Que ▁cette ▁prop os it ion , ▁que ▁la ▁N ature ▁ab h or re ▁le ▁v ui de , ▁et ▁n é an m oins ▁l ' ad m et , ▁l ' ac c use ▁d ' i mp uis s ance , ▁ou ▁imp l ique ▁contr ad ict ion 

In [38]:
outputs_postproc = decode_sp(outputs)

In [39]:
ecrire_fichier(outputs_postproc, 'data/dev.sp.norm.trg')